In [ ]:
from can.interface import Bus
from cantools.database import load_file

In [ ]:
import sys
import os
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '\\..\\..')
print(sys.path)
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from broqer import Sink
import ipywidgets as widgets

In [ ]:
from canbro import Node

In [ ]:
db = load_file('device_CAN_e2e.dbc')
db.nodes

In [ ]:
db.messages[0].dbc.attributes['DataIds'].value

In [ ]:
db.messages[0].signals[0].dbc.attributes['GenSigFuncType'].value

In [ ]:
bus_e= Bus('test', interface='virtual')
ecu = Node(name="ECU",bus=bus_e,database=db )

In [ ]:
bus_v= Bus('test', interface='virtual')
vcu= Node(name='CONTROL',bus=bus_v,database=db )  

In [ ]:
label_value = widgets.Label(value="value=?")
def show_vcu_value(value, label: widgets.Label):
    label.value = 'value={}'.format(value)

In [ ]:
show_print = ecu.DEM._signal_DEM_SNC.subscribe(Sink(show_vcu_value, label=label_value))

In [ ]:
vcu.DEM._signal_operation_mode._metadata.initial

In [ ]:
vcu.DEM._signal_operation_mode._metadata.initial=0

In [ ]:
ecu.DEM._signal_operation_mode.notify(0)

In [ ]:
import ipywidgets as widgets
import time

def update_ecu_value():
    for i in range(6):
        vcu.DEM._signal_operation_mode.notify(i)
        print(f'show Count: {i}')
        time.sleep(20)
    vcu.DEM._signal_operation_mode.notify("Standby")
    print(f'show Count: Standby')

def start_task(button):
    button.description = 'is running'
    update_ecu_value()
    

button = widgets.Button(description='Start')
button.on_click(start_task)

display(button)

In [ ]:
display(label_value)

Label(value='value=1')

DEBUG:root:send periodic e2e send message DEM at timestep 1698077716.533109
DEBUG:root:Update signals: DEM
DEBUG:root:Signal lv_set changed to value = 83
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = Standby
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = 221
DEBUG:root:Signal lv_set changed to value = 13.0


In [ ]:
vcu.DEM._signal_operation_mode.notify(3)

DEBUG:root:Update CAN message: DEM
DEBUG:root:Update CAN Data: {'DEM_CRC': 154, 'DEM_SNC': 1, 'operation_mode': 3, 'li_set': 1, 'hv_set': 221, 'lv_set': 13.0}
DEBUG:root:update periodic e2e send message DEM


DEBUG:root:send periodic e2e send message DEM at timestep 1698077722.43283
DEBUG:root:Update signals: DEM
DEBUG:root:Signal lv_set changed to value = 154
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = Pre-charge
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = 221
DEBUG:root:Signal lv_set changed to value = 13.0
DEBUG:root:send periodic e2e send message DEM at timestep 1698077726.551155
DEBUG:root:Update signals: DEM
DEBUG:root:Signal lv_set changed to value = 154
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = Pre-charge
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = 221
DEBUG:root:Signal lv_set changed to value = 13.0
DEBUG:root:send periodic e2e send message DEM at timestep 1698077727.847841
DEBUG:root:Update signals: DEM
DEBUG:root:Signal lv_set changed to value = 154
DEBUG:root:Signal lv_set changed to value = 1
DE

In [ ]:
vcu.DEM.start_periodic()

DEBUG:root:send periodic e2e send message DEM at timestep 1698077706.527397
DEBUG:root:Update signals: DEM
DEBUG:root:Signal lv_set changed to value = 83
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = Standby
DEBUG:root:Signal lv_set changed to value = 1
DEBUG:root:Signal lv_set changed to value = 221
DEBUG:root:Signal lv_set changed to value = 13.0


In [ ]:
type(vcu.DEM)

In [ ]:
type(ecu.DEM)